In [ ]:
import gym
import tensorflow as tf
import tensorflow.compat.v1 as tfc
import numpy as np
import random
from collections import deque
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
import random
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Hyper Parameters
GAMMA = 0.95 # discount factor
LEARNING_RATE=0.01
bound = np.vstack((np.zeros((30)),np.ones((30))))
tfc.disable_eager_execution()

In [ ]:
# data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NOx-216.csv',index_col='Time') # 11 features
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NOx-Data/MIC_0.15_in_regression.csv',index_col='Time') # 30 features
df= data.values
df[:,1:] = (df[:,1:]-df[:,1:].min(axis=0))/(df[:,1:].max(axis=0)-df[:,1:].min(axis=0))
X_train, X_test, y_train, y_test = train_test_split(df[:,1:],df[:,0], test_size=0.2)

model = GradientBoostingRegressor(min_samples_split=18, max_depth=16, learning_rate=0.1)
model.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=16, min_samples_split=18)

In [ ]:
class NOXEnv(gym.Env):
  def __init__(self):
    self.action_space = gym.spaces.Discrete(60)
    self.observation_space = gym.spaces.Box(low=0, high=1, shape=(30,1), dtype=np.float32)

  def reset(self):
    temp = random.randint(0,X_test.shape[0]-1)
    self.state = X_test[temp,:]
    print(y_test[temp])
    return self.state

  def step(self, state, action):
    reward = 0.0
    done = False

    if action%2 == 0: # 偶数为增加，奇数为减少，action的范围是[0,21]
      state[int(action/2)] = state[int(action/2)] + 0.1*(bound[1, int(action/2)] - state[int(action/2)]) # increase action
    else:
      state[int(action/2)] = state[int(action/2)] - 0.1*(state[int(action/2)] - bound[0, int(action/2)]) # decrease action

    self.state = state

    output = model.predict(state[np.newaxis,:])
    # reward = 3*state[5] + 2*state[2] - state[0] - state[1] - output[0]/10 # consumed NOx + actual feed - ammonia - furnace temperature - boiler pressure
    reward = -output[0]

    if output[0] <= 150:
      done = True

    return self.state, reward, done, output[0]

In [ ]:
class Actor():
    def __init__(self, env, sess):
        # init some parameters
        self.time_step = 0
        self.state_dim = env.observation_space.shape[0]
        self.action_dim = env.action_space.n
        self.create_softmax_network()

        # Init session
        self.session = sess
        self.session.run(tfc.global_variables_initializer())

    def create_softmax_network(self):
        # network weights
        W1 = self.weight_variable([self.state_dim, 20])
        b1 = self.bias_variable([20])
        W2 = self.weight_variable([20, self.action_dim])
        b2 = self.bias_variable([self.action_dim])
        # input layer
        self.state_input = tfc.placeholder("float", [None, self.state_dim])
        self.tf_acts = tfc.placeholder(tf.int32, [None,60], name="actions_num")
        self.td_error = tfc.placeholder(tf.float32, None, "td_error")  # TD_error
        # hidden layers
        h_layer = tf.nn.relu(tf.matmul(self.state_input, W1) + b1)
        # softmax layer
        self.softmax_input = tf.matmul(h_layer, W2) + b2
        # softmax output
        self.all_act_prob = tf.nn.softmax(self.softmax_input, name='act_prob')

        self.neg_log_prob = tf.nn.softmax_cross_entropy_with_logits(logits=self.softmax_input,
                                                                           labels=self.tf_acts)
        self.exp = tf.reduce_mean(self.neg_log_prob * self.td_error)

        #这里需要最大化当前策略的价值，因此需要最大化self.exp,即最小化-self.exp
        self.train_op = tfc.train.AdamOptimizer(LEARNING_RATE).minimize(-self.exp)

    def weight_variable(self, shape):
        initial = tfc.truncated_normal(shape)
        return tf.Variable(initial)

    def bias_variable(self, shape):
        initial = tf.constant(0.01, shape=shape)
        return tf.Variable(initial)

    def choose_action(self, observation):
        prob_weights = self.session.run(self.all_act_prob, feed_dict={self.state_input: observation[np.newaxis, :]})
        action = np.random.choice(range(prob_weights.shape[1]), p=prob_weights.ravel())  # select action w.r.t the actions prob
        return action

    def learn(self, state, action, td_error):
        s = state[np.newaxis, :]
        one_hot_action = np.zeros(self.action_dim)
        one_hot_action[action] = 1
        a = one_hot_action[np.newaxis, :]
        # train on episode
        self.session.run(self.train_op, feed_dict={
             self.state_input: s,
             self.tf_acts: a,
             self.td_error: td_error,
        })

In [ ]:
EPSILON = 0.01 # final value of epsilon
REPLAY_SIZE = 10000 # experience replay buffer size
BATCH_SIZE = 32 # size of minibatch
REPLACE_TARGET_FREQ = 10 # frequency to update target Q network

In [ ]:
class Critic():
    def __init__(self, env, sess):
        # init some parameters
        self.time_step = 0
        self.epsilon = EPSILON
        self.state_dim = env.observation_space.shape[0]
        self.action_dim = env.action_space.n

        self.create_Q_network()
        self.create_training_method()

        # Init session
        self.session = sess
        self.session.run(tfc.global_variables_initializer())

    def create_Q_network(self):
        #network weights
        W1q = self.weight_variable([self.state_dim, 20])
        b1q = self.bias_variable([20])
        W2q = self.weight_variable([20, 1])
        b2q = self.bias_variable([1])
        self.state_input = tfc.placeholder(tf.float32, [1, self.state_dim], "state")
        # hidden layers
        h_layerq = tfc.nn.relu(tf.matmul(self.state_input, W1q) + b1q)
        # Q Value layer
        self.Q_value = tfc.matmul(h_layerq, W2q) + b2q

      # def create_Q_network(self, scope, trainable):
      #   with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):

      #     self.state_input = tfc.placeholder(tf.float32, [1, self.state_dim], "state")

      #     # 卷积层，32个8×8卷积，步长为4，全零填充，激活函数为ReLU
      #     h_conv1 = tf.keras.layers.Conv2D(filters=32, kernel_size=[8,8],strides=4,padding="same",activation=tf.nn.relu,kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.01),\
      #                                       bias_initializer=tf.constant_initializer(0.01),trainable=trainable)(self.state_input)
          
      #     # 池化层，最大池化，窗口大小为2×2，步长为2
      #     h_pool1 = tf.keras.layers.MaxPooling2D(pool_size=[2,2],strides=2, padding="SAME")(h_conv1)
          
      #     # 卷积层，64个4×4卷积，步长为2，全零填充，激活函数为ReLU
      #     h_conv2 = tf.keras.layers.Conv2D(filters=64, kernel_size=[4,4],strides=2, padding="same", activation=tf.nn.relu, kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.01),\
      #                                     bias_initializer=tf.constant_initializer(0.01),trainable=trainable)(h_pool1)
          
      #     # 卷积层，64个3×3卷积，步长为1，全零填充，激活函数为ReLU
      #     h_conv3 = tf.keras.layers.Conv2D(filters=64, kernel_size=[3,3],strides=1,padding="same",activation=tf.nn.relu, kernel_initializer=tf.random_normal_initializer(mean=0,stddev=0.01),\
      #                                     bias_initializer=tf.constant_initializer(0.01),trainable=trainable)(h_conv2)
          
      #     # 扁平化
      #     h_conv3_flat = tf.keras.layers.Flatten()(h_conv3)
          
      #     # 全连接层，512个神经元，激活函数为ReLU
      #     h_fc1 = tf.keras.layers.Dense(units=512,activation=tf.nn.relu, kernel_initializer=tf.random_normal_initializer(0,stddev=0.01),\
      #                                   bias_initializer=tf.constant_initializer(0.01),trainable=trainable)(h_conv3_flat)
          
      #     # 输出层，2个神经元，表示两个动作分别对应的预测Q值，没有激活函数
      #     self.Q_value = tf.keras.layers.Dense(units=22,kernel_initializer=tf.random_normal_initializer(0,stddev=0.01),\
      #                                   bias_initializer=tf.constant_initializer(0.01),trainable=trainable)(h_fc1)



    def create_training_method(self):
        self.next_value = tfc.placeholder(tf.float32, [1,1], "v_next")
        self.reward = tfc.placeholder(tf.float32, None, 'reward')

        with tfc.variable_scope('squared_TD_error'):
            self.td_error = self.reward + GAMMA * self.next_value - self.Q_value
            self.loss = tfc.square(self.td_error)
        with tfc.variable_scope('train'):
            self.train_op = tfc.train.AdamOptimizer(self.epsilon).minimize(self.loss)

    def train_Q_network(self, state, reward, next_state):
        s, s_ = state[np.newaxis, :], next_state[np.newaxis, :]
        v_ = self.session.run(self.Q_value, {self.state_input: s_})
        td_error, _ = self.session.run([self.td_error, self.train_op],
                                          {self.state_input: s, self.next_value: v_, self.reward: reward})
        return td_error

    def weight_variable(self,shape):
        initial = tfc.truncated_normal(shape)
        return tf.Variable(initial)

    def bias_variable(self,shape):
        initial = tfc.constant(0.01, shape = shape)
        return tf.Variable(initial)

In [ ]:
# Hyper Parameters
EPISODE = 1000 # Episode limitation
STEP = 5000 # Step limitation in an episode
TEST = 10 # The number of experiment test every 100 episode

def main():
  # initialize OpenAI Gym env and dqn agent
  sess = tfc.InteractiveSession()
  env = NOXEnv()
  actor = Actor(env, sess)
  critic = Critic(env, sess)
  result_reward = []
  result_step = []

  for episode in range(EPISODE):
    # initialize task
    state = env.reset()
    total_reward = 0
    # Train
    for step in range(STEP):
      action = actor.choose_action(state) # e-greedy action for train
      next_state,reward,done,output = env.step(state, action)
      td_error = critic.train_Q_network(state, reward, next_state)  # gradient = grad[r + gamma * V(s_) - V(s)]
      actor.learn(state, action, td_error)  # true_gradient = grad[logPi(s,a) * td_error]
      state = next_state
      total_reward += reward
      ave_reward = total_reward/(step+1)
      if done or step == STEP-1:
        result_reward.append(ave_reward)
        result_step.append(step+1)
        break
    print(reward,step,output)


    # Test every 100 episodes
    # if episode % 10 == 0:
    #   total_reward = 0
    #   for i in range(TEST):
    #     state = env.reset()
    #     reward1 = []
    #     for j in range(STEP):
    #       action = actor.choose_action(state) # direct action for test
    #       state,reward,done,_ = env.step(state, action)
    #       total_reward += reward
    #       reward1.append(reward)
    #       if done:
    #         break
      
    #   ave_reward = total_reward/TEST
    #   print ('episode: ',episode,'Evaluation Average Reward:',ave_reward)
      # print('reward1: ',reward1)
      # plt.plot(reward1)
      # plt.show()

  visualization1(result_reward, result_step)

def visualization1(reward, step):
  fig = plt.figure(figsize=(10,6))
  ax1 = fig.add_subplot(111)
  ax1.plot(range(1,len(reward)+1),reward,linestyle='dashed',label='average reward')
  ax1.set_xticks([0,20,40,60,80,100])
  ax2 = ax1.twinx()
  ax2.plot(range(1,len(step)+1),step,color='r',label='time step')


if __name__ == '__main__':
  main()

/usr/local/lib/python3.7/dist-packages/tensorflow/python/client/session.py:1768: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


198.43518
-199.6618211996385 4999 199.6618211996385
182.72223
-185.7021194590272 4999 185.7021194590272
192.73148
-189.7658059243592 4999 189.7658059243592
225.92592
-228.47693847035814 4999 228.47693847035814
189.46297
-189.37111867475357 4999 189.37111867475357
225.42593
-225.72611636672647 4999 225.72611636672647
199.65741
-205.77637723057325 4999 205.77637723057325
202.69444
-199.5573596531857 4999 199.5573596531857
185.7037
-184.91548025715954 4999 184.91548025715954
191.40741
-190.99387011921593 4999 190.99387011921593
219.18518
-216.16503606402256 4999 216.16503606402256
212.66667
-209.53431231659997 4999 209.53431231659997
198.18518
-193.61457402662307 4999 193.61457402662307
212.44444
-213.47023230140522 4999 213.47023230140522
153.19444
-153.55423436411712 4999 153.55423436411712
224.70369
-217.93660165380078 4999 217.93660165380078
179.97221
-178.40772425850804 4999 178.40772425850804
208.90741
-203.35992469627888 4999 203.35992469627888
181.0
-180.23715623455738 4999 180.23